# `.log` — Universal Drill-Down

One accessor at every level. The chain is always `.log.steps[i].log.steps[j]...`

| Object | `.log` returns | Type |
|--------|---------------|------|
| `RunResult` | trace of this run | `RunLog` |
| `MapResult` | batch overview | `MapLog` |
| `NodeRecord` (leaf) | — | `None` |
| `NodeRecord` (1 inner) | inner trace | `RunLog` |
| `NodeRecord` (N inner) | batch of inner traces | `MapLog` |

In [ ]:
from hypergraph import SyncRunner, Graph, node, ifelse, END


@node(output_name="doubled")
def double(x: int) -> int:
    return x * 2


@node(output_name="tripled")
def triple(doubled: int) -> int:
    return doubled * 3


runner = SyncRunner()
graph = Graph([double, triple], name="pipeline")

## 1. Single run → `RunLog`

`result.log` — just type it, the table shows automatically.

In [ ]:
result = runner.run(graph, {"x": 5})
result.log

In [ ]:
result.log.summary()

In [ ]:
result.log.timing

In [ ]:
result.log.node_stats

Leaf nodes have `.log = None` — nothing nested:

In [ ]:
result.log.steps[0].log is None

## 2. Mapped run → `MapLog`

`results.log` gives you a `MapLog` — batch overview with per-item drill-down.

In [ ]:
results = runner.map(graph, {"x": [1, 2, 3, 4, 5]}, map_over="x")
results.log

In [ ]:
results.log.summary()

Drill into one item — **same `RunLog` as Step 1**:

In [ ]:
results.log[0]

In [ ]:
# Aggregate stats across ALL items (cross-item bottleneck analysis)
results.log.node_stats

## 3. Nested graph → `.log` on steps

When a graph runs as a node inside another graph, `step.log` reveals the inner trace.

In [ ]:
inner = Graph([double, triple], name="pipeline")
outer = Graph([inner.as_node()])
result = runner.run(outer, {"x": 5})
result.log

The footer tells you where to drill. Single nested → `step.log` is a `RunLog`:

In [ ]:
result.log.steps[0].log

### map_over → `step.log` is a `MapLog`

When the inner graph runs N times, `step.log` returns a `MapLog` instead:

In [ ]:
outer_mapped = Graph([inner.as_node().map_over("x")])
result = runner.run(outer_mapped, {"x": [1, 2, 3, 4, 5]})
result.log

In [ ]:
step = result.log.steps[0]
step.log  # MapLog — same as results.log in Step 2

In [ ]:
step.log[0]  # drill into first item → RunLog

## 4. Deep nesting — the chain keeps going

Three levels deep: outer → middle → innermost. The `.log` chain works at every level.

In [ ]:
@node(output_name="incremented")
def increment(doubled: int) -> int:
    return doubled + 1


innermost = Graph([double], name="innermost")
middle = Graph([innermost.as_node(), increment], name="middle")
outer = Graph([middle.as_node()])

result = runner.run(outer, {"x": 5})
result.log

In [ ]:
# outer → middle (RunLog)
middle_log = result.log.steps[0].log
middle_log

In [ ]:
# middle → innermost (RunLog)
innermost_step = next(s for s in middle_log.steps if s.node_name == "innermost")
innermost_step.log

## 5. Errors — find failures across items

`MapLog.errors` aggregates all failed `NodeRecord`s across all items.

In [ ]:
@node(output_name="result")
def maybe_fail(x: int) -> int:
    if x % 2 == 0:
        raise ValueError(f"even: {x}")
    return x * 10


fail_graph = Graph([maybe_fail], name="checker")
results = runner.map(fail_graph, {"x": [1, 2, 3, 4, 5]}, map_over="x", error_handling="continue")
results.log

In [ ]:
results.log.errors

In [ ]:
# Which items failed?
[(i, log.errors[0].error) for i, log in enumerate(results.log) if log.errors]

## 6. Serialization

Both `RunLog` and `MapLog` serialize to JSON via `.to_dict()`.

In [ ]:
import json

result = runner.run(graph, {"x": 5})
print(json.dumps(result.log.to_dict(), indent=2))

In [ ]:
results = runner.map(graph, {"x": [1, 2]}, map_over="x")
print(json.dumps(results.log.to_dict(), indent=2))